## TuneTA Research

In [1]:
import os
from pathlib import Path


# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "/Users/liuqiaowei/SourceCode/freqtrade/user_data/notebooks"
i = 0
try:
    os.chdir(project_root)
    if not Path("LICENSE").is_file():
        i = 0
        while i < 4 and (not Path("LICENSE").is_file()):
            os.chdir(Path(Path.cwd(), "../"))
            i += 1
        project_root = Path.cwd()
except FileNotFoundError:
    print("Please define the project root relative to the current directory")
print(Path.cwd())

/Users/liuqiaowei/SourceCode/freqtrade


In [2]:
from freqtrade.configuration import Configuration


# Customize these according to your needs.

# Initialize empty configuration object
config = Configuration.from_files(["./user_data/config.json"])
# Optionally (recommended), use existing configuration file
# config = Configuration.from_files(["user_data/config.json"])

# Location of the data
data_location = config["datadir"]
config['timeframe'] = '1h'
# Pair to analyze - Only use one pair here
# config['exchange']['pair_whitelist']

In [3]:
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from pandas_ta import percent_return

candles_dict = {}
for pair in config['exchange']['pair_whitelist']:
    candles = load_pair_history(
        datadir=data_location,
        timeframe=config["timeframe"],
        pair=pair,
        data_format="feather",  # Make sure to update this to your data
        candle_type=CandleType.FUTURES,
    )
    candles['sym'] = pair
    candles.set_index('date', inplace=True)
    candles.set_index('sym', append=True, inplace=True)
    candles['return'] = percent_return(candles.close, offset=-4)
    # Keep Latest week for testing
    candles_dict[pair] = candles


In [4]:
candles_dict['DASH/USDT:USDT']

,,open,high,low,close,volume,return
date,sym,,,,,,
2024-03-17 20:00:00+00:00,DASH/USDT:USDT,37.55,37.58,37.19,37.29,12375.246,-0.028226
2024-03-17 21:00:00+00:00,DASH/USDT:USDT,37.28,37.39,37.17,37.32,15350.452,0.011895
2024-03-17 22:00:00+00:00,DASH/USDT:USDT,37.33,37.76,37.26,37.28,29419.310,0.008475
2024-03-17 23:00:00+00:00,DASH/USDT:USDT,37.28,37.37,36.99,37.20,12950.961,0.000542
2024-03-18 00:00:00+00:00,DASH/USDT:USDT,37.20,37.24,36.14,36.15,26128.211,0.006231
...,...,...,...,...,...,...,...
2024-11-12 21:00:00+00:00,DASH/USDT:USDT,26.19,26.32,25.65,26.28,26259.807,-0.006100
2024-11-12 22:00:00+00:00,DASH/USDT:USDT,26.28,26.72,26.08,26.68,11730.460,NaN
2024-11-12 23:00:00+00:00,DASH/USDT:USDT,26.67,26.71,26.18,26.24,13785.907,NaN


In [5]:
from tuneta.tune_ta import TuneTA
import pandas as pd

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [6]:
X = pd.concat([candles for _, candles in candles_dict.items()], axis=0).sort_index()
X.dropna(inplace=True)
y = X['return']
X = X.drop(columns=['return'])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, shuffle=False)

In [8]:
tt = TuneTA(n_jobs=4, verbose=True)

In [9]:
tt.fit(X_train, y_train,
        indicators=['tta.ADX', 'tta.ADXR', 'tta.APO', 'tta.AROON', 'tta.AROONOSC', 'tta.BOP', 'tta.CCI', 'tta.CMO', 'tta.DX', 'tta.MACD', 'tta.MACDEXT', 'tta.MACDFIX', 'tta.MFI', 'tta.MINUS_DI', 'tta.MINUS_DM', 'tta.MOM', 'tta.PLUS_DI', 'tta.PLUS_DM', 'tta.PPO', 'tta.ROC', 'tta.ROCP', 'tta.ROCR', 'tta.ROCR100', 'tta.RSI', 'tta.STOCH', 'tta.STOCHF', 'tta.STOCHRSI', 'tta.TRIX', 'tta.ULTOSC','tta.WILLR', 'tta.AD', 'tta.ADOSC', 'tta.OBV', 'tta.ATR', 'tta.NATR', 'tta.TRANGE'],
        ranges=[(4, 30), (31, 60)],
        trials=300,
        early_stop=50,
)
  
tt.fit_times()

[I 2024-11-13 10:24:53,463] A new study created in memory with name: tta.ADX(X.high, X.low, X.close, timeperiod=trial.suggest_int('timeperiod', 4, 30), )
[I 2024-11-13 10:24:53,507] A new study created in memory with name: tta.ADXR(X.high, X.low, X.close, timeperiod=trial.suggest_int('timeperiod', 4, 30), )
[I 2024-11-13 10:24:53,554] A new study created in memory with name: tta.APO(X.close, fastperiod=trial.suggest_int('fastperiod', 4, 30), slowperiod=trial.suggest_int('slowperiod', 4, 30), )
[I 2024-11-13 10:24:53,594] A new study created in memory with name: tta.AROON(X.high, X.low, timeperiod=trial.suggest_int('timeperiod', 4, 30), )
[I 2024-11-13 10:24:56,190] Trial 0 finished with value: 0.018221742270311262 and parameters: {'timeperiod': 14}. Best is trial 0 with value: 0.018221742270311262.
[I 2024-11-13 10:24:56,291] Trial 0 finished with value: 0.06018479363345729 and parameters: {'timeperiod': 14}. Best is trial 0 with value: 0.06018479363345729.
[I 2024-11-13 10:24:56,344] 

    Indicator       Times
--  ------------  -------
54  tta.STOCHRSI   688.48
29  tta.STOCH      592.11
42  tta.STOCHF     518.89
13  tta.PPO        428.35
56  tta.MACD       393.17
35  tta.ULTOSC     388.22
36  tta.ADOSC      357.09
30  tta.STOCH      325.02
55  tta.MACDEXT    317.86
52  tta.STOCHF     310.29
71  tta.STOCHRSI   308.81
 2  tta.PPO        303.31
61  tta.MACDEXT    294.08
60  tta.MACDFIX    288.48
44  tta.WILLR      256.21
23  tta.CMO        255.5
32  tta.PLUS_DI    245.45
58  tta.MACD       241.81
39  tta.ADOSC      241.6
22  tta.RSI        241.21
14  tta.DX         237.81
26  tta.ULTOSC     228
 3  tta.ROC        220.12
 4  tta.ROCP       218.7
 6  tta.ROCR100    216.8
 5  tta.ROCR       214.6
41  tta.ATR        210.22
34  tta.CCI        207.4
17  tta.ADXR       205.58
27  tta.MFI        205.2
20  tta.MINUS_DI   202.74
64  tta.APO        201.62
45  tta.MINUS_DM   195.15
16  tta.ADX        189.8
18  tta.ADXR       189.29
59  tta.MOM        188.99
63  tta.MOM        187.

In [10]:
tt.report(target_corr=True, features_corr=False)


Indicator Correlation to Target:

                                                              Correlation
----------------------------------------------------------  -------------
tta_NATR_timeperiod_11                                           0.18893
tta_NATR_timeperiod_40                                           0.179893
tta_PPO_fastperiod_29_slowperiod_23                              0.082435
tta_ROC_timeperiod_28                                            0.08241
tta_ROCP_timeperiod_28                                           0.08241
tta_ROCR_timeperiod_28                                           0.08241
tta_ROCR100_timeperiod_28                                        0.08241
tta_ROCP_timeperiod_52                                           0.082376
tta_ROC_timeperiod_52                                            0.082376
tta_ROCR_timeperiod_52                                           0.082376
tta_ROCR100_timeperiod_52                                        0.082376
tta_TRIX

In [11]:
tt.prune(max_inter_correlation=.8)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [12]:
tt.report(target_corr=True, features_corr=True)


Indicator Correlation to Target:

                                                             Correlation
---------------------------------------------------------  -------------
tta_NATR_timeperiod_11                                          0.18893
tta_PPO_fastperiod_29_slowperiod_23                             0.082435
tta_TRIX_timeperiod_34                                          0.081988
tta_PPO_fastperiod_44_slowperiod_32                             0.078873
tta_DX_timeperiod_55                                            0.074857
tta_ADX_timeperiod_34                                           0.072415
tta_MINUS_DI_timeperiod_39                                      0.063193
tta_ULTOSC_timeperiod1_57_timeperiod2_59_timeperiod3_60         0.048094
tta_MFI_timeperiod_35                                           0.045243
tta_STOCH_fastk_period_32_slowk_period_57_slowd_period_55       0.04137
tta_STOCH_fastk_period_18_slowk_period_29_slowd_period_5        0.040906
tta_AROON_timeperi

In [13]:
tt.t_corr[tt.t_corr['Correlation'] > 0.02]

,Correlation
tta_NATR_timeperiod_11,0.188930
tta_PPO_fastperiod_29_slowperiod_23,0.082435
tta_TRIX_timeperiod_34,0.081988
tta_PPO_fastperiod_44_slowperiod_32,0.078873
tta_DX_timeperiod_55,0.074857
tta_ADX_timeperiod_34,0.072415
tta_MINUS_DI_timeperiod_39,0.063193
tta_ULTOSC_timeperiod1_57_timeperiod2_59_timeperiod3_60,0.048094
tta_MFI_timeperiod_35,0.045243
tta_STOCH_fastk_period_32_slowk_period_57_slowd_period_55,0.041370


In [14]:
tt.t_corr.to_csv('./tune_ta_result.csv')

In [15]:
features = tt.transform(X_train)

In [16]:
features.columns

Index(['tta_NATR_timeperiod_11', 'tta_PPO_fastperiod_29_slowperiod_23',
       'tta_TRIX_timeperiod_34', 'tta_PPO_fastperiod_44_slowperiod_32',
       'tta_DX_timeperiod_55', 'tta_ADX_timeperiod_34',
       'tta_MINUS_DI_timeperiod_39',
       'tta_ULTOSC_timeperiod1_57_timeperiod2_59_timeperiod3_60',
       'tta_MFI_timeperiod_35',
       'tta_STOCH_fastk_period_32_slowk_period_57_slowd_period_55_0',
       'tta_STOCH_fastk_period_32_slowk_period_57_slowd_period_55_1',
       'tta_STOCH_fastk_period_18_slowk_period_29_slowd_period_5_0',
       'tta_STOCH_fastk_period_18_slowk_period_29_slowd_period_5_1',
       'tta_AROON_timeperiod_44_0', 'tta_AROON_timeperiod_44_1',
       'tta_PLUS_DI_timeperiod_56', 'tta_ADOSC_fastperiod_24_slowperiod_5',
       'tta_AROONOSC_timeperiod_28', 'tta_ADOSC_fastperiod_31_slowperiod_36',
       'tta_PLUS_DM_timeperiod_58',
       'tta_STOCHF_fastk_period_56_fastd_period_59_0',
       'tta_STOCHF_fastk_period_56_fastd_period_59_1',
       'tta_WILLR_time